# Разные способы получить ключевые слова

Так получилось, что я делала похожее задание раньше, так что я просто сдам вам эту тетрадку. Тут неправильно реализован индекс Жаккара и некоторые вещи я бы сейчас реализовала лучше, но, к сожалению, времени или сил править что-либо в нет. 

### Подготовка тестовых данных

Получаем тексты

In [ ]:
import os

In [ ]:
PATH = 'samples'

In [ ]:
def get_texts(path):
    """берет путь к папке с текстами и извлекает тексты в виде строк"""
    texts = []
    for file in os.listdir(path):
        if file.endswith('.txt'):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as f:
                texts.append(f.read())
    return texts

In [ ]:
texts = get_texts(PATH)

Получаем лемматизированные с помощью MyStem тексты без стоп-слов

In [ ]:
import re
from pymystem3 import Mystem
m = Mystem()
from stop_words import get_stop_words

In [ ]:
def get_mystem_lemmatized_text(text):
    """
    берет текст в виде строки и возвращает его 
    лемматизированным MyStem в виде списка слов
    """
    stop_words = set(get_stop_words('ru'))
    stop_words.update(['чей', 'свой', 'ежели', 'нешто', 'из-за'])
    lemmatized_text = [word for word in m.lemmatize(text)
                       if re.search('[а-яёa-z]', word)
                       and word not in stop_words]
    return lemmatized_text

In [ ]:
mystem_lemmatized_texts = [get_mystem_lemmatized_text(text) for text in texts]

Получаем лемматизированные с помощью pymorphy2 тексты без стоп-слов

In [ ]:
import re
from string import punctuation 
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk import word_tokenize

In [ ]:
def get_pymorphy2_lemmatized_text(text):
    """
    берет текст в виде строки и возвращает его без стоп-слов и
    лемматизированным pymorphy2 в виде списка слов
    """
    stop_words = set(get_stop_words('ru'))
    stop_words.update(['чей', 'свой', 'ежели', 'нешто', 'из-за'])
    lemmatized_text = [morph.parse(word)[0].normal_form.strip(punctuation+'.— ') 
                       for word in word_tokenize(text)
                       if re.search('[а-яёa-z]', word, flags=re.I)
                       and morph.parse(word)[0].normal_form.strip(punctuation+'.— ') 
                       not in stop_words]
    return lemmatized_text

In [ ]:
pymorphy2_lemmatized_texts = [get_pymorphy2_lemmatized_text(text) for text in texts]

### 7 способов извлечения ключевых слов

Все способы подразумевают применение на лемматизированном тексте без стоп-слов

#### 1. Ключевые слова по частотности

In [ ]:
from collections import Counter

In [ ]:
def get_keywords_most_frequent(text, num_keywords):
    """
    берет текст в виде списка слов и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов самые частотные слова текста
    """
    keywords = [word_freq[0] for word_freq in Counter(text).most_common(num_keywords)]
    return keywords

In [ ]:
for text in pymorphy2_lemmatized_texts:
    print(get_keywords_most_frequent(text, 5))

['гайка', 'денис', 'грузило', 'понимать', 'следователь']
['иван', 'пугачев', 'савельй', 'отвечать', 'марья']
['колобок', 'уйти', 'короб', 'сусек', 'заяц']
['тест', 'тестирование', 'лаборатория', 'роспотребнадзор', 'коронавирус']
['неделя', 'расход', 'россиянин', 'категория', 'товар']


In [ ]:
for text in mystem_lemmatized_texts:
    print(get_keywords_most_frequent(text, 5))

['денис', 'гайка', 'отвинчивать', 'грузило', 'понимать']
['пугачев', 'марья', 'ивановна', 'савельич', 'отвечать']
['колобок', 'уходить', 'лиса', 'короб', 'сусек']
['тестирование', 'лаборатория', 'тест', 'роспотребнадзор', 'коронавирус']
['неделя', 'расход', 'россиянин', 'категория', 'товар']


#### 2. Ключевые слова и биграммы по частотности

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def get_keywords_bigrams_most_frequent(text, num_keywords):
    """
    берет текст в виде списка слов и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов самые частотные биграммы текста, 
    согласованные грамматически, и самые частотные слова 
    в зависимости от их нормированной частотности
    """
    bigrams = [(text[i], text[i + 1]) for i, word in enumerate(text) if i != (len(text) - 1)]
    freq_uninflected_bigrams_numbers = dict([bigram_freq for bigram_freq in Counter(bigrams).most_common(num_keywords)])
    freq_words_numbers = dict([(word_freq[0], word_freq[1] / 2) for word_freq in Counter(text).most_common(num_keywords)])
    freq_items_numbers = {**freq_uninflected_bigrams_numbers , **freq_words_numbers}
    freq_items_numbers = sorted(freq_items_numbers.items(), key=lambda item: item[1], reverse=True)[:num_keywords]
    keywords = []
    for item in freq_items_numbers:
        if type(item[0]) == tuple: 
            new_bigram = None
            first_word = morph.parse(item[0][0])[0]
            second_word = morph.parse(item[0][1])[0]
            if 'ADJF' in first_word.tag or 'ADJS' in first_word.tag:
                if second_word.tag.gender:
                    new_bigram = (first_word.inflect({second_word.tag.gender}).word, item[0][1])
            elif 'tran' in first_word.tag:
                if 'NOUN' in second_word.tag or 'ADJF' in first_word.tag or 'ADJS' in first_word.tag:
                    new_bigram = (item[0][0], second_word.inflect({'accs'}).word)
            elif 'intr' in first_word.tag and 'NOUN' in second_word.tag:
                if 'NOUN' in second_word.tag or 'ADJF' in first_word.tag or 'ADJS' in first_word.tag:
                    new_bigram = (item[0][0], second_word.inflect({'ablt'}).word)
            elif 'NOUN' in first_word.tag and 'NOUN' in second_word.tag:
                if 'Surn' not in first_word.tag and 'Name' not in first_word.tag:
                    new_bigram = (item[0][0], second_word.inflect({'gent'}).word)
            if new_bigram:
                keywords.append(' '.join(new_bigram))
            else:
                keywords.append(' '.join(item[0]))
        else:
             keywords.append(item[0]) 
    return keywords

In [ ]:
for text in pymorphy2_lemmatized_texts:
    print(get_keywords_bigrams_most_frequent(text, 5))

['гайка', 'денис', 'грузило', 'понимать', 'следователь']
['иван', 'марья иван', 'пугачев', 'иван кузмич', 'савельй']
['колобок', 'уйти', 'колобок колобка', 'сказка колобка', 'колобок картинки']
['тест', 'тестирование', 'лаборатория', 'роспотребнадзор', 'тестирование коронавируса']
['неделя', 'салон красоты', 'расход', 'сократить расход', 'ювелирное изделие']


In [ ]:
for text in mystem_lemmatized_texts:
    print(get_keywords_bigrams_most_frequent(text, 5))

['денис', 'гайка', 'отвинчивать гайку', 'отвинчивать', 'грузило']
['марья ивановна', 'пугачев', 'марья', 'ивановна', 'иван кузмич']
['колобок', 'уходить', 'колобок колобка', 'сказка колобка', 'колобок картинки']
['тестирование', 'лаборатория', 'тест', 'роспотребнадзор', 'тестирование коронавируса']
['неделя', 'салон красоты', 'расход', 'сокращать расход', 'ювелирное изделие']


#### 3. Ключевые слова и биграммы по частотности и без глаголов

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def get_keywords_bigrams_most_frequent_without_verbs(text, num_keywords):
    """
    берет текст в виде списка слов и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов самые частотные биграммы текста, 
    согласованные грамматически и не содержащие глаголы, и самые частотные слова, 
    не являющиеся глаголами, в зависимости от их нормированной частотности
    """
    bigrams = [(text[i], text[i + 1]) for i, word in enumerate(text) if i != (len(text) - 1)]
    freq_uninflected_bigrams_numbers = dict([bigram_freq for bigram_freq in Counter(bigrams).most_common()])
    freq_words_numbers = dict([(word_freq[0], word_freq[1] / 2) for word_freq in Counter(text).most_common()])
    freq_items_numbers = list({**freq_uninflected_bigrams_numbers , **freq_words_numbers}.items())
    items_to_remove = []
    for item in freq_items_numbers:
        if type(item[0]) == str:
            word = morph.parse(item[0])[0]
            if 'INFN' in word.tag:
                items_to_remove.append(item)
        else: 
            first_word = morph.parse(item[0][0])[0]
            second_word = morph.parse(item[0][1])[0]
            if 'INFN' in first_word.tag or 'INFN' in second_word.tag:
                items_to_remove.append(item)
    freq_items_numbers = [item for item in freq_items_numbers if item not in items_to_remove]
    freq_items_numbers = sorted(freq_items_numbers, key=lambda item: item[1], reverse=True)[:num_keywords]
    keywords = []
    for item in freq_items_numbers:
        if type(item[0]) == tuple: 
            new_bigram = None
            first_word = morph.parse(item[0][0])[0]
            second_word = morph.parse(item[0][1])[0]
            if 'ADJF' in first_word.tag or 'ADJS' in first_word.tag:
                if second_word.tag.gender:
                    new_bigram = (first_word.inflect({second_word.tag.gender}).word, item[0][1])
            elif 'NOUN' in first_word.tag and 'NOUN' in second_word.tag:
                if second_word.inflect({'gent'}):
                    if 'Surn' not in first_word.tag and 'Name' not in first_word.tag:
                        new_bigram = (item[0][0], second_word.inflect({'gent'}).word)
            if new_bigram:
                keywords.append(' '.join(new_bigram))
            else:
                keywords.append(' '.join(item[0]))
        else:
             keywords.append(item[0]) 
    return keywords

In [ ]:
for text in pymorphy2_lemmatized_texts:
    print(get_keywords_bigrams_most_frequent_without_verbs(text, 5))

['гайка', 'денис', 'грузило', 'следователь', 'рельс']
['иван', 'марья иван', 'пугачев', 'иван кузмич', 'савельй']
['колобок', 'колобок колобка', 'сказка колобка', 'колобок картинки', 'колобок короба']
['тест', 'тестирование', 'лаборатория', 'роспотребнадзор', 'тестирование коронавируса']
['неделя', 'салон красоты', 'расход', 'ювелирное изделие', 'уровень потребления']


In [ ]:
for text in mystem_lemmatized_texts:
    print(get_keywords_bigrams_most_frequent_without_verbs(text, 5))

['денис', 'гайка', 'грузило', 'следователь', 'рельс']
['марья ивановна', 'пугачев', 'марья', 'ивановна', 'иван кузмич']
['колобок', 'колобок колобка', 'сказка колобка', 'колобок картинки', 'колобок короба']
['тестирование', 'лаборатория', 'тест', 'роспотребнадзор', 'тестирование коронавируса']
['неделя', 'салон красоты', 'расход', 'ювелирное изделие', 'аналитик сбербанка']


#### 4. Ключевые слова по TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def get_keywords_tf_idf(texts, num_keywords):
    """
    берет список текстов в виде строк или списков
    и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов слова, отобранные методом TF-IDF
    """
    keywords = []
    make_tf_idf = TfidfVectorizer()
    for text in texts:
        if type(text) == list:
            texts_as_tfidf_vectors = make_tf_idf.fit_transform(' '.join(text) for text in texts)
        elif type(text) == str:
            texts_as_tfidf_vectors = make_tf_idf.fit_transform(texts)
    id2word = {i:word for i, word in enumerate(make_tf_idf.get_feature_names())} 
    for text_row in range(texts_as_tfidf_vectors.shape[0]): 
        row_data = texts_as_tfidf_vectors.getrow(text_row) 
        words_for_this_text = row_data.toarray().argsort() 
        top_words_for_this_text = words_for_this_text [0,:-(num_keywords+1):-1] 
        keywords_for_this_text = [id2word[w] for w in top_words_for_this_text]
        keywords.append(keywords_for_this_text)
    return keywords

In [ ]:
for text_keywords in get_keywords_tf_idf(pymorphy2_lemmatized_texts, 5):
    print(text_keywords)

['гайка', 'денис', 'грузило', 'следователь', 'понимать']
['иван', 'пугачев', 'марья', 'савельй', 'крепость']
['колобок', 'уйти', 'короб', 'заяц', 'сусек']
['тест', 'тестирование', 'роспотребнадзор', 'лаборатория', 'коронавирус']
['неделя', 'расход', 'товар', 'россиянин', 'категория']


In [ ]:
for text_keywords in get_keywords_tf_idf(mystem_lemmatized_texts, 5):
    print(text_keywords)

['денис', 'гайка', 'грузило', 'отвинчивать', 'понимать']
['пугачев', 'марья', 'ивановна', 'савельич', 'крепость']
['колобок', 'уходить', 'лиса', 'сусек', 'короб']
['тест', 'тестирование', 'роспотребнадзор', 'лаборатория', 'коронавирус']
['неделя', 'расход', 'товар', 'категория', 'россиянин']


#### 5. Ключевые слова и биграммы по TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words
from pymystem3 import Mystem
m = Mystem()

In [ ]:
rus_stop_words = get_stop_words('ru')
rus_stop_words.extend(['чей', 'свой', 'ежели', 'нешто', 'из-за'])

In [ ]:
def get_preprocessed_text(text):
    """
    берет текст в виде строки и возвращает его без стоп-слов и
    лемматизированным pymorphy2 в виде списка слов
    """
    preprocessed_text = ' '.join([morph.parse(word)[0].normal_form.strip(punctuation+'.— ') 
                       for word in word_tokenize(text)
                       if re.search('[а-яёa-z]', word, flags=re.I)])
    return preprocessed_text

In [ ]:
def get_keywords_bigrams_tf_idf(texts, num_keywords):
    """
    берет список не лемматизированных текстов в виде строк
    и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов слова и биграммы, 
    отобранные методом TF-IDF
    """
    keywords = []
    make_tf_idf = TfidfVectorizer(stop_words=rus_stop_words, ngram_range=(1, 2))
    texts_as_tfidf_vectors = make_tf_idf.fit_transform(get_preprocessed_text(text) for text in texts)
    id2word = {i : word for i, word in enumerate(make_tf_idf.get_feature_names())} 
    for text_row in range(texts_as_tfidf_vectors.shape[0]): 
        row_data = texts_as_tfidf_vectors.getrow(text_row) 
        words_for_this_text = row_data.toarray().argsort() 
        top_words_for_this_text = words_for_this_text [0, : -(num_keywords + 1) : -1] 
        keywords_for_this_text = [id2word[w] for w in top_words_for_this_text]
        keywords.append(keywords_for_this_text)
    return keywords

In [ ]:
for text_keywords in get_keywords_bigrams_tf_idf(texts, 5):
    print(text_keywords)

['гайка', 'денис', 'грузило', 'следователь', 'понимать']
['иван', 'пугачев', 'марья', 'савельй', 'марья иван']
['колобок', 'уйти', 'колобок колобок', 'короб', 'лис']
['тест', 'тестирование', 'роспотребнадзор', 'лаборатория', 'коронавирус']
['неделя', 'расход', 'россиянин', 'товар', 'категория']


#### 6. Ключевые слова по degree centrality графа совместной встречаемости слов в тексте

In [ ]:
import networkx as nx

In [ ]:
def get_keywords_graph_degree_centrality(text, num_keywords):
    """
    берет текст в виде списка слов и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов узлы графа с самой высокой степенью degree centrality
    """
    G = nx.Graph()
    G.add_edges_from([(text[i], text[i+1]) for i, item in enumerate(text) if i != (len(text)-1)])
    dc = nx.degree_centrality(G)
    keywords = [node for node in sorted(dc, key=dc.get, reverse=True)[:num_keywords]]
    return keywords

In [ ]:
for text in pymorphy2_lemmatized_texts:
    print(get_keywords_graph_degree_centrality(text, 5))

['гайка', 'денис', 'грузило', 'понимать', 'следователь']
['пугачев', 'иван', 'савельй', 'отвечать', 'швабрин']
['колобок', 'лис', 'старуха', 'уйти', 'волк']
['тест', 'лаборатория', 'роспотребнадзор', 'тестирование', 'биоматериал']
['неделя', 'расход', 'россиянин', 'категория', 'товар']


In [ ]:
for text in mystem_lemmatized_texts:
    print(get_keywords_graph_degree_centrality(text, 5))

['денис', 'гайка', 'грузило', 'понимать', 'отвинчивать']
['пугачев', 'савельич', 'отвечать', 'крепость', 'становиться']
['колобок', 'лиса', 'старуха', 'уходить', 'волк']
['лаборатория', 'тест', 'роспотребнадзор', 'тестирование', 'биоматериал']
['неделя', 'расход', 'россиянин', 'категория', 'товар']


#### 7. Ключевые слова по betweenness centrality графа совместной встречаемости слов в тексте

In [ ]:
import networkx as nx

In [ ]:
def get_keywords_graph_betweenness_centrality(text, num_keywords):
    """
    берет текст в виде списка слов и количество ключевых слов для извлечения,
    возвращает в качестве ключевых слов узлы графа с самой высокой степенью degree centrality
    """
    G = nx.Graph()
    G.add_edges_from([(text[i], text[i+1]) for i, item in enumerate(text) if i != (len(text)-1)])
    bc = nx.betweenness_centrality(G)
    keywords = [node for node in sorted(bc, key=bc.get, reverse=True)[:num_keywords]]
    return keywords

In [ ]:
for text in pymorphy2_lemmatized_texts:
    print(get_keywords_graph_betweenness_centrality(text, 5))

['гайка', 'денис', 'грузило', 'следователь', 'понимать']
['пугачев', 'иван', 'савельй', 'отвечать', 'швабрин']
['колобок', 'старуха', 'далёкий', 'короб', 'уйти']
['лаборатория', 'тест', 'роспотребнадзор', 'коронавирус', 'тестирование']
['неделя', 'расход', 'сбербанк', 'россиянин', 'категория']


In [ ]:
for text in mystem_lemmatized_texts:
    print(get_keywords_graph_betweenness_centrality(text, 5))

['денис', 'грузило', 'гайка', 'понимать', 'следователь']
['пугачев', 'савельич', 'отвечать', 'крепость', 'швабрин']
['колобок', 'старуха', 'лиса', 'уходить', 'короб']
['лаборатория', 'роспотребнадзор', 'тест', 'частный', 'тестирование']
['неделя', 'расход', 'сбербанк', 'трата', 'категория']


### Оценка способов

In [ ]:
import json

Возьмем один из доступных корпусов с метаданными в JSON

In [ ]:
def get_corpus(corpus_name):
    """
    берет в формате строки название корпуса, существующего в формате jsonlines, 
    и возвращает его в виде списка словарей
    """
    with open(f'data/{corpus_name}.jsonlines', 'r', encoding='utf-8') as f:
        corpus_name = [json.loads(line) for line in f]
        return corpus_name

In [ ]:
corpus = get_corpus('russia_today_7')

Извлекаем все тексты корпуса в список, где каждый текст -- строка

In [ ]:
corpus_texts = [item['content'] for item in corpus if item['content']] 
print(f'{len(corpus_texts)} текстов')
print(f'{sum(len(text.split()) for text in corpus_texts)} слов')

218 текстов
207852 слов


Для каждого текста корпуса достаем написанные человеком т.н. эталонные ключевые слова

In [ ]:
manual_keywords = [item['keywords'] for item in corpus if item['content']] 
print(f'{len(manual_keywords)} групп ключевых слов')

218 групп ключевых слов


Начинаем сравнивать их с т.н. предсказанными ключевыми словами 

Вначале лемматизируем двумя способами корпус текстов

In [ ]:
pymorphy2_lemmatized_corpus_texts = [get_pymorphy2_lemmatized_text(text) for text in corpus_texts]

In [ ]:
mystem_lemmatized_corpus_texts = [get_mystem_lemmatized_text(text) for text in corpus_texts]

Применяем все способы (в огромной функции, чтобы можно было менять количество запрашиваемых ключевых слов)

In [ ]:
def get_all_methods_keywords(num_keywords):
    """
    берет количество запрашиваемых ключевых слов в наборе
    и возвращает списки наборов ключевых слов
    """
    #1
    pymorphy2_predicted_keywords_most_frequent = [get_keywords_most_frequent(text, num_keywords)
                                                  for text in pymorphy2_lemmatized_corpus_texts]
    #2
    pymorphy2_predicted_keywords_bigrams_most_frequent = [get_keywords_bigrams_most_frequent(text, num_keywords)
                                                          for text in pymorphy2_lemmatized_corpus_texts]
    #3
    pymorphy2_predicted_keywords_bigrams_most_frequent_without_verbs = [get_keywords_bigrams_most_frequent_without_verbs
                                                                        (text, num_keywords) for text in 
                                                                        pymorphy2_lemmatized_corpus_texts]
    #4
    pymorphy2_predicted_keywords_tf_idf = get_keywords_tf_idf(pymorphy2_lemmatized_corpus_texts, num_keywords)
    #5
    pymorphy2_predicted_keywords_graph_degree_centrality = [get_keywords_graph_degree_centrality(text, num_keywords)
                                                            for text in pymorphy2_lemmatized_corpus_texts]
    #6
    pymorphy2_predicted_keywords_graph_betweenness_centrality = [get_keywords_graph_betweenness_centrality
                                                                 (text, num_keywords) for text in 
                                                                 pymorphy2_lemmatized_corpus_texts]
    #7 
    mystem_predicted_keywords_most_frequent = [get_keywords_most_frequent(text, num_keywords) 
                                               for text in mystem_lemmatized_corpus_texts]
    #8
    mystem_predicted_keywords_bigrams_most_frequent = [get_keywords_bigrams_most_frequent(text, num_keywords) 
                                                       for text in mystem_lemmatized_corpus_texts]
    #9
    mystem_predicted_keywords_bigrams_most_frequent_without_verbs = [get_keywords_bigrams_most_frequent_without_verbs
                                                                     (text, num_keywords)
                                                                     for text in mystem_lemmatized_corpus_texts]
    #10
    mystem_predicted_keywords_tf_idf = get_keywords_tf_idf(mystem_lemmatized_corpus_texts, num_keywords)
    #11
    mystem_predicted_keywords_graph_degree_centrality = [get_keywords_graph_degree_centrality(text, num_keywords) 
                                                         for text in mystem_lemmatized_corpus_texts]
    #12
    mystem_predicted_keywords_graph_betweenness_centrality = [get_keywords_graph_betweenness_centrality
                                                              (text, num_keywords) for text in 
                                                              mystem_lemmatized_corpus_texts]
    #13
    predicted_keywords_bigrams_tf_idf = get_keywords_bigrams_tf_idf(corpus_texts, num_keywords)
    return pymorphy2_predicted_keywords_most_frequent, \
           pymorphy2_predicted_keywords_bigrams_most_frequent,\
           pymorphy2_predicted_keywords_bigrams_most_frequent_without_verbs, \
           pymorphy2_predicted_keywords_tf_idf, \
           pymorphy2_predicted_keywords_graph_degree_centrality,\
           pymorphy2_predicted_keywords_graph_betweenness_centrality,\
           mystem_predicted_keywords_most_frequent,\
           mystem_predicted_keywords_bigrams_most_frequent,\
           mystem_predicted_keywords_bigrams_most_frequent_without_verbs,\
           mystem_predicted_keywords_tf_idf,\
           mystem_predicted_keywords_graph_degree_centrality,\
           mystem_predicted_keywords_graph_betweenness_centrality,\
           predicted_keywords_bigrams_tf_idf

In [ ]:
pymorphy2_predicted_keywords_most_frequent, \
pymorphy2_predicted_keywords_bigrams_most_frequent,\
pymorphy2_predicted_keywords_bigrams_most_frequent_without_verbs, \
pymorphy2_predicted_keywords_tf_idf, \
pymorphy2_predicted_keywords_graph_degree_centrality,\
pymorphy2_predicted_keywords_graph_betweenness_centrality,\
mystem_predicted_keywords_most_frequent,\
mystem_predicted_keywords_bigrams_most_frequent,\
mystem_predicted_keywords_bigrams_most_frequent_without_verbs,\
mystem_predicted_keywords_tf_idf,\
mystem_predicted_keywords_graph_degree_centrality,\
mystem_predicted_keywords_graph_betweenness_centrality,\
predicted_keywords_bigrams_tf_idf = get_all_methods_keywords(12)

Чтобы получить названия вызываемых функций для наглядности рейтинга

In [ ]:
from inspect import currentframe

In [ ]:
def get_name_of_function_returned_var(var):
    """берет переменную и возвращает название функции, которая ее произвела"""
    callers_local_vars = currentframe().f_back.f_back.f_locals.items()
    name_of_function_returned_var = [var_name for var_name, var_val in callers_local_vars if var_val is var][0]
    return name_of_function_returned_var

Функция оценки качества предсказания:

In [ ]:
def get_evaluation(manual_keywords, predicted_keywords):
    """
    функция оценки, берет списки наборов эталонных ключевых слов и предсказанных
    ключевых слов, возвращает название метода и в кортеже его точность, 
    полноту, F-меру и коэффициент Жаккара предсказания 
    """
    method_name = get_name_of_function_returned_var(predicted_keywords)
    precisions = []
    recalls = []
    intersections = []
    for index, one_text_manual_keywords in enumerate(manual_keywords):
        one_text_predicted_keywords = predicted_keywords[index]
        intersection = len(set(one_text_manual_keywords) & set(one_text_predicted_keywords))
        intersections.append(intersection)
        recalls.append(intersection / len(one_text_manual_keywords)) 
        precisions.append(intersection / len(one_text_predicted_keywords))
    num_manual_keywords = sum(len(one_text_manual_keywords) 
                              for one_text_manual_keywords in manual_keywords)
    num_predicted_keywords = sum(len(one_text_predicted_keywords) 
                                 for one_text_predicted_keywords in predicted_keywords)
    num_intersections = sum(intersections) 
    mean_precision = sum(precisions) / len(precisions)
    mean_recall = sum(recalls) / len(recalls)
    fmeasure =  (2 * mean_recall * mean_precision) / (mean_recall + mean_precision)
    jaccard_index =  num_intersections / (num_manual_keywords + num_predicted_keywords + num_intersections)
    return method_name, (mean_precision, mean_recall, fmeasure, jaccard_index)

Применяем функцию оценки на всех полученных списках ключевых слов (по 13 наборов ключевых слов для каждого текста корпуса)

In [ ]:
evaluations = []
#1 
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_most_frequent))
#2
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_bigrams_most_frequent))
#3
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_bigrams_most_frequent_without_verbs))
#4
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_tf_idf))
#5
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_graph_degree_centrality))
#6
evaluations.append(get_evaluation(manual_keywords, pymorphy2_predicted_keywords_graph_betweenness_centrality))
#7
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_most_frequent))
#8
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_bigrams_most_frequent))
#9
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_bigrams_most_frequent_without_verbs))
#10
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_tf_idf))
#11
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_graph_degree_centrality))
#12
evaluations.append(get_evaluation(manual_keywords, mystem_predicted_keywords_graph_betweenness_centrality))
#13
evaluations.append(get_evaluation(manual_keywords, predicted_keywords_bigrams_tf_idf))

Отсортируем оценки по некоторым аспектам, для этого напишем функцию выдачи рейтинга по номеру аспекта

In [ ]:
def get_aspect_rating(evaluations, aspect_number):
    """
    берет список оценок в формате dict.items() 
    и номер аспекта, где
    0 -- точность, 1 -- полнота,
    2 -- F-мера, 3 -- коэффициент Жаккара,
    и врзвращает рейтинг способов по убыванию оценки этого аспекта
    """
    rating = sorted(evaluations, key=lambda item: item[1][aspect_number], reverse=True)
    best = rating[0][0]
    worst = rating[-1][0]
    rating_to_print = []
    for index, item in enumerate(rating):
        mean_precision, mean_recall, fmeasure, jaccard_index = item[1]
        rating_to_print.append(f'{index + 1}. {item[0]}'
                               f'\nТочность: {mean_precision}\nПолнота: {mean_recall}\n'
                               f'F-мера: {fmeasure}\nКоэффициент Жаккара: {jaccard_index}\n')
    rating_to_print = '\n'.join(rating_to_print)
    return rating_to_print, best, worst

In [ ]:
rating_to_print, best, worst = get_aspect_rating(evaluations, 2)

Рейтинг способов по F-мере:

In [ ]:
print(rating_to_print)

1. pymorphy2_predicted_keywords_graph_degree_centrality
Точность: 0.1567278287461774
Полнота: 0.19446103630911435
F-мера: 0.1735673253288406
Коэффициент Жаккара: 0.0763615946097698

2. mystem_predicted_keywords_graph_degree_centrality
Точность: 0.15405198776758414
Полнота: 0.19190860725169337
F-мера: 0.17090907370641228
Коэффициент Жаккара: 0.07514992503748126

3. pymorphy2_predicted_keywords_most_frequent
Точность: 0.1521406727828746
Полнота: 0.18837074452582325
F-мера: 0.16832828708817393
Коэффициент Жаккара: 0.0742824985931345

4. mystem_predicted_keywords_most_frequent
Точность: 0.14984709480122324
Полнота: 0.18702227863226484
F-мера: 0.1663834549903541
Коэффициент Жаккара: 0.07323943661971831

5. pymorphy2_predicted_keywords_graph_betweenness_centrality
Точность: 0.14487767584097855
Полнота: 0.17967175985369196
F-мера: 0.16040962712595944
Коэффициент Жаккара: 0.07097138554216867

6. mystem_predicted_keywords_bigrams_most_frequent_without_verbs
Точность: 0.14373088685015292
Полнота

Для интереса можно посмотреть на первые несколько эталонных, лучших предсказанных и худших предсказанных групп ключевых слов

In [ ]:
def print_manual_best_worst_keywords(best, worst, num_keywords_groups):
    """
    берет списки лучших и худших ключевых слов и количество групп ключевых слов,
    печатает их вместе с эталонными ключевыми словами
    """
    for one_text_manual, one_text_best, one_text_worst in zip(manual_keywords[:num_keywords_groups], 
                                                              globals()[best][:num_keywords_groups], 
                                                              globals()[worst][:num_keywords_groups]):
        print(f'Эталонные слова: {one_text_manual}'
              f'\nЛучший метод {best}: {one_text_best}'
              f'\nХудший метод {worst}: {one_text_worst}\n')

In [ ]:
print_manual_best_worst_keywords(best, worst, 3)

Эталонные слова: ['выборы в сша', 'сша', 'санкции', 'хакеры', 'эксклюзив rt', 'демократическая партия ', 'дональд трамп', 'закон', 'кибератака', 'кибербезопасность', 'политика', 'республиканская партия', 'хиллари клинтон', 'сша']
Лучший метод pymorphy2_predicted_keywords_graph_degree_centrality: ['штат', 'россия', 'сша', 'кибератака', 'резолюция', 'иллинойс', 'трамп', 'система', 'сервер', 'избирательный', 'атака', 'отметить']
Худший метод predicted_keywords_bigrams_tf_idf: ['штат', 'иллинойс', 'резолюция', 'штат иллинойс', 'избирательный', 'кибератака', 'сервер', 'избирательный комиссия', 'взлом', 'проект резолюция', 'комиссия', 'комиссия штат']

Эталонные слова: ['гражданство', 'ес', 'латвия', 'национализм', 'права человека', 'прибалтика', 'россия', 'россияне', 'русофобия ', 'русский язык', 'ссср', 'прибалтика']
Лучший метод pymorphy2_predicted_keywords_graph_degree_centrality: ['латвия', 'гражданство', 'негражданин', 'ребёнок', 'латвийский', 'государство', 'право', 'паспорт', 'страна

### Выводы

1. MyStem работает дольше pymorphy2 в гораздо большей степени чем лучше
2. На разных корпусах результаты могут сильно отличаться, поэтому нельзя сказать однозначно, какой способ лучший и худший
3. Функции по частотности и по degree centrality очень эффективные при своей простоте и скорости выполнения
4. На этом корпусе текстов моя собственная функция (наиболее авторской я считаю "Ключевые слова и биграммы по частотности и без глаголов") показала хороший результат, обогнав TF-IDF с биграммами и без
5. Идея убирать глаголы из предсказываемых ключевых слов достаточно эффективная
6. TF-IDF с биграммами вероятно будет эффективнее, если прикрутить грамматическое согласование и убрать глаголы
7. Возможно, разница между лучшим и худшим способом в 3% -- это не так и много